# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import os
import zipfile
import pandas as pd

from azureml.core import Workspace, Dataset, Experiment

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-152157
aml-quickstarts-152157
southcentralus
610d6e37-4747-4a20-80eb-3aad70a55f43


In [5]:
# Create compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
dataset = Dataset.get_by_name(ws, name='Ames-housing-dataset')
print(dataset.to_pandas_dataframe().info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 no

In [6]:
# Choose a name for experiment
experiment_name = 'Ames-housing-AutoML'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_settings = {
    'experiment_timeout_minutes' : 120,
    'n_cross_validations' : 3,
    'enable_early_stopping' : True,
    'iteration_timeout_minutes' : 5,
    'max_concurrent_iterations' : 4,
    'max_cores_per_iteration' : -1
}

automl_config = AutoMLConfig(
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    compute_target=cpu_cluster,
    training_data=dataset,
    label_column_name='SalePrice',
    **automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_0fc9eea6-09ad-40ac-9ec1-b571665bbd87,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Column name              

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
# Get your best run and save the model from that run.
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
print('Best Run Id: ', best_run.id, '\n')
best_run_metrics

Best Run Id:  AutoML_0fc9eea6-09ad-40ac-9ec1-b571665bbd87_36 



{'mean_absolute_percentage_error': 9.189548385041606,
 'r2_score': 0.8867183587869617,
 'normalized_mean_absolute_error': 0.02152348548410304,
 'explained_variance': 0.8868079393249927,
 'normalized_root_mean_squared_error': 0.037018768677643726,
 'spearman_correlation': 0.956256808492198,
 'root_mean_squared_log_error': 0.13271070134427843,
 'mean_absolute_error': 15499.061897102598,
 'normalized_root_mean_squared_log_error': 0.04316913222805933,
 'median_absolute_error': 9696.481985777713,
 'root_mean_squared_error': 26657.21532477124,
 'normalized_median_absolute_error': 0.013465465887762413,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_0fc9eea6-09ad-40ac-9ec1-b571665bbd87_36/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_0fc9eea6-09ad-40ac-9ec1-b571665bbd87_36/predicted_true'}

In [16]:
import json
with open('best_run_metrics.json', 'w') as file:
    json.dump(best_run_metrics, file)
with open('best_run_details.txt', 'w') as file:
    file.write(str(best_run.get_details()))

In [18]:
# Check the path to the model
best_run.get_file_names()[-6] # where is output/model.joblib ?[-4]? CHECK it in the next cell

'outputs/model.pkl'

In [35]:
# Save the best model
from azureml.core import Model
os.makedirs('./outputs/', exist_ok=True)
best_run.download_file(best_run.get_file_names()[-6], output_file_path='./outputs/')
model = Model.register(ws, model_path='outputs/model.pkl', model_name='Ames-Housing-AutoML-Model', tags=best_run_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-AutoML-Model
Ames-Housing-AutoML-Model	Ames-Housing-AutoML-Model:2	2


In [44]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-152157', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-152157'), name=Ames-Housing-AutoML-Model, id=Ames-Housing-AutoML-Model:2, version=2, tags={'mean_absolute_percentage_error': '9.189548385041606', 'r2_score': '0.8867183587869617', 'normalized_mean_absolute_error': '0.02152348548410304', 'explained_variance': '0.8868079393249927', 'normalized_root_mean_squared_error': '0.037018768677643726', 'spearman_correlation': '0.956256808492198', 'root_mean_squared_log_error': '0.13271070134427843', 'mean_absolute_error': '15499.061897102598', 'normalized_root_mean_squared_log_error': '0.04316913222805933', 'median_absolute_error': '9696.481985777713', 'root_mean_squared_error': '26657.21532477124', 'normalized_median_absolute_error': '0.013465465887762413', 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_0fc9eea6-09ad-40ac-9ec1-b571665bbd87_36/residuals', 'predicted_true': 'aml://artifact

In [45]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Ames-housing-AutoML,AutoML_0fc9eea6-09ad-40ac-9ec1-b571665bbd87_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli#script

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [46]:
# Delete() is used to deprovision and delete the AmlCompute target. 
cpu_cluster.delete()